In [1]:
#import libraries
import os
import numpy as np
import pandas as pd
import psycopg2

In [2]:
#find csv files in current working directory
os.listdir(os.getcwd())

['Pulse_May_2020_survey_public_data.csv',
 '.DS_Store',
 'Pulse_Aug_2020_survey_public_data.csv',
 'csv_import_automation.ipynb',
 'pulse_2018_import.ipynb',
 'Pulse_2018_survey_public_data.csv',
 '.ipynb_checkpoints',
 'Pulse_2019_survey_public_data.csv',
 'Pulse_2021_survey_public_data.csv']

In [3]:
#isolate only csv files with a for loop
csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [4]:
#make a new directory
dataset_dir = 'datasets'
#create bash command to make a new dir
#mkdir dataset_dir

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

In [5]:
#move the csv files to the new directory

#mv filename directory
for csv in csv_files:
    mv_file = "mv '{0}' {1}".format(csv, dataset_dir)
    os.system(mv_file)
    print(mv_file)


mv 'Pulse_May_2020_survey_public_data.csv' datasets
mv 'Pulse_Aug_2020_survey_public_data.csv' datasets
mv 'Pulse_2018_survey_public_data.csv' datasets
mv 'Pulse_2019_survey_public_data.csv' datasets
mv 'Pulse_2021_survey_public_data.csv' datasets


In [6]:
#dataframe
data_path = os.getcwd()+'/'+dataset_dir+'/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1", low_memory=False)
    print(file)

/var/folders/yr/hk4ztld11c53hnlmp45js6000000gn/T/ipykernel_28987/2555810783.py:7: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df[file] = pd.read_csv(data_path+file)
/var/folders/yr/hk4ztld11c53hnlmp45js6000000gn/T/ipykernel_28987/2555810783.py:7: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df[file] = pd.read_csv(data_path+file)


Pulse_May_2020_survey_public_data.csv
Pulse_Aug_2020_survey_public_data.csv
Pulse_2018_survey_public_data.csv
Pulse_2019_survey_public_data.csv
Pulse_2021_survey_public_data.csv


/var/folders/yr/hk4ztld11c53hnlmp45js6000000gn/T/ipykernel_28987/2555810783.py:7: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df[file] = pd.read_csv(data_path+file)


In [7]:
# clean table names
# lowercase, replace whitespace with _, replace symbols with _

for k in csv_files:
    dataframe = df[k]

    clean_table_name = k.lower().replace(" ", "_").replace("?", "") \
                    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
                    .replace(")", "").replace(r"(","").replace("$","")
  
    
    #remove .csv extention from clean table name
    tbl_name = '{0}'.format(clean_table_name.split('.')[0])
    print(tbl_name)
    
    dataframe.columns = [x.lower().replace(" ", "_").replace("?", "") \
                    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
                    .replace(")", "").replace(r"(","").replace("$","") for x in dataframe.columns]
  
    
    #create replacements dictionary that maps panda dtypes to sql dtypes
    replacements = {
    'object' : 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]':'varchar'
    }
    
    #create the create table schema - join name and datatype, replacing the pandas dtype with sql datatype as loop
    col_string = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements)))
   
    
    #open database connection to postgres using psycopg2
    host = 'localhost'
    dbname = 'FHN'
    user = 'user'
    password = 'password'
    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)

    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')

    #drop tables with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))

    #CREATE TABLE
    cursor.execute("create table %s (%s);" % (tbl_name, col_string))
    print('{0} was created successfully'.format(tbl_name))

    #insert values to table

    #save df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')

    #open csv files, save as an object
    my_file = open(k)
    print('file opened in memory')

    #upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')

    conn.commit()
    cursor.close
    print('table {0} imported to db completed'.format(tbl_name))

    #for loop end message
    print('all tables have been successfully imported into the database')

pulse_may_2020_survey_public_data
opened database successfully
pulse_may_2020_survey_public_data was created successfully
file opened in memory
file copied to db
table pulse_may_2020_survey_public_data imported to db completed
all tables have been successfully imported into the database
pulse_aug_2020_survey_public_data
opened database successfully
pulse_aug_2020_survey_public_data was created successfully
file opened in memory
file copied to db
table pulse_aug_2020_survey_public_data imported to db completed
all tables have been successfully imported into the database
pulse_2018_survey_public_data
opened database successfully
pulse_2018_survey_public_data was created successfully
file opened in memory
file copied to db
table pulse_2018_survey_public_data imported to db completed
all tables have been successfully imported into the database
pulse_2019_survey_public_data
opened database successfully
pulse_2019_survey_public_data was created successfully
file opened in memory
file copied 